In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import scanpy as sc
import anndata
from tqdm import trange

In [ ]:
for cas in trange(1, 21):
	adata = anndata.read_h5ad(f"../../simulation/sample/sample{cas}.h5ad")
	print(adata)
	bat = adata.obs[["Batch"]].copy()
	rawdat = adata.X#.toarray()
	rawcorr = torch.corrcoef(torch.tensor(rawdat))

	dat = rawdat
	bat = bat.iloc[:, 0]
	mx = bat.value_counts().idxmax()
	ref_block = dat[bat == mx]
	ref_corr = torch.corrcoef(torch.tensor(ref_block))
	dattype = bat.unique()

	corr = rawcorr
	vec = ref_corr.reshape(-1).sort()[0]
	for i in dattype:
		for j in dattype:
			if i == mx and j == mx:
				continue
			block = corr[bat == i][:, bat == j]
			block_ = block.reshape(-1)
			idx = block_.argsort()
			pos = torch.zeros_like(idx, dtype=torch.float)
			pos[idx] = torch.arange(len(idx), dtype=torch.float)
			pos = pos / len(block_) * len(vec)
			pos = torch.maximum(pos.int() - 1, torch.zeros_like(pos, dtype=torch.int))
			block_ = vec[pos]
			block_ = block_.reshape(block.shape)
			
			msk = (torch.tensor(bat == i, dtype=torch.bool).unsqueeze(1) &
				   torch.tensor(bat == j, dtype=torch.bool).unsqueeze(0))
			corr.masked_scatter_(msk, block_)

	for epoch in range(1):
		batmsk = {i: (bat != i).to_numpy() for i in dattype}
		for i in trange(len(corr)):
			curbat = batmsk[bat[i]]
			block = corr[i, curbat]
			idx = block.argsort().int()
			pos = torch.zeros_like(idx, dtype=torch.float)
			pos[idx] = torch.arange(len(idx), dtype=torch.float)
			pos = pos / len(block) * len(vec)
			pos = torch.maximum(pos.int() - 1, torch.zeros_like(pos, dtype=torch.int))
			block = vec[pos]
			msk = torch.zeros_like(corr, dtype=torch.bool)
			msk[i] |= torch.tensor(curbat, dtype=torch.bool)
			msk = msk.bool()
			corr.masked_scatter_(msk, block)
		corr = (corr + corr.T) / 2

	pd.DataFrame(corr.numpy()).to_csv(f"sample/sample{cas}.csv")